In [182]:
import re
import string as stri
import requests
from xlrd import open_workbook
import xlwt

In [43]:
results = getInfo("802_16_Completed_Projects.xls")
output = []
files = set()
total = len(results)
for idx, result in zip(range(total), results):
    print("%.2f%%"%((idx+1)*100/total), end='\r')
    url = result[7]  # file link
    filename = url[url.rfind('/')+1:]
    if filename in files:
        continue
    files.add(filename)
    getFileFromUrl("clean data/80216CompProj/", url, filename)
    result[5] = getName(result[4])  # author name, generated from title
    result.append(filename)
    output.append(result)

In [48]:
len(output)

8441

In [256]:
def getName(string):
    author = ""
    if (string.rfind('(') != -1 and string.rfind(')') != -1):
        end = string.rfind(')')
        start = getStartEnd(string, end)
        content = string[start+1:end]
        tokens = re.split(';\s*|,\s*|\s+and\s+', content)
        for token in tokens:
            token = token.strip()
            mat1 = re.search(r"(\d{4}-\d{2}-\d{2})", token)
            mat2 = re.search(r"(\d{2}/\d{2}/\d{2})", token)
            mat3 = re.search(r"(\d{4}-\d{2})", token)
            if mat1 is not None or mat2 is not None or mat3 is not None:
                continue
            token = token.replace('et al.', '')
            token = token.replace('et. al', '')
            token = token.replace('et al', '')
            token = token.strip()
            if token != "":
                author += token + ";"
    return author

In [247]:
p = re.compile(r'et[.*] al[.*]')
m=p.match("Inseok Hwang et. al.")

In [242]:
m.group()

AttributeError: 'NoneType' object has no attribute 'group'

In [254]:
#Number of pilots in OFDM FFT=256 mode with UL subchanelization (Naftali Chayat,02/11/14)
author = getName("Corrections for PRBS Operation in AAS Zone with Diversity Map Support (Inseok Hwang et al)")

In [255]:
author

'Inseok Hwang;'

In [117]:
def getStartEnd(string, end):
    cnt = 0
    for idx in range(end-1, -1, -1):
        if string[idx] == ')':
            cnt += 1
        elif string[idx] == '(' and cnt == 0:
            break
        elif string[idx] == '(':
            cnt -= 1    
    return idx    

In [37]:
def getInfo(filename):
    book = open_workbook(filename, on_demand=True)
    book.sheet_names()
    results = []
    for name in book.sheet_names():
        sheet = book.sheet_by_name(name)
        for row in range(1, sheet.nrows):
            result = []
            for cell in sheet.row(row):
                result.append(cell.value)
            results.append(result)
    return results

In [40]:
def getFileFromUrl(path, url, filename):
    response = requests.get(url, allow_redirects=True)
    with open(path + filename, 'wb') as file:
        file.write(response.content)

In [258]:
filename = "clean data/802_16_Completed_Projects.xls"
book = xlwt.Workbook(encoding="utf-8")
sheet1 = book.add_sheet("Sheet 1")
sheet1.write(0, 0, "Feature1")
sheet1.write(0, 1, "Feature2")
sheet1.write(0, 2, "Tag Text")
sheet1.write(0, 3, "Session Info")
sheet1.write(0, 4, "Title")
sheet1.write(0, 5, "Author")
sheet1.write(0, 6, "Time")
sheet1.write(0, 7, "File Link")
sheet1.write(0, 8, "Additional Infoamtion")
sheet1.write(0, 9, "Filename")

In [259]:
# [feature1, feature2, tagtext, sessionInfo, title, author, time, fileLink, addiInfo]
for idx1, line in zip(range(1, len(output)+1), output):
    for idx2, val in zip(range(0, len(line)), line):
        sheet1.write(idx1, idx2, val)

In [260]:
book.save(filename)

In [257]:
results = getInfo("clean data/802_16_Completed_Projects.xls")
output = []
total = len(results)
for idx, result in zip(range(total), results):
    print("%.2f%%"%((idx+1)*100/total), end='\r')
    result[5] = getName(result[4])  # author name, generated from title
    output.append(result)